In [1]:
import random
import os

import neat
from neat import DistributedEvaluator, ParallelEvaluator
from tqdm.notebook import tqdm, trange
from pathlib import Path
from random import randint
from ui.python.Layout import Layout
import numpy as np

from helpers.estimation_helpers import *
import helpers.visualize as visualize

import pandas as pd

In [2]:
MAX_WORKERS = 10
SLICE_SIZE = 400
EPOCHS = 100
SCORE_COEFFICIENTS = (500, 350, 150)
layout = Layout('./../data/layouts/genetic/step1-max/layout_4_racks.json').reset_item_count().reset_path_count()
item_list = None
check_list = None
check_ids = None
df = None
best = None
str_item = None
selected_categories = [
 'bakery',
 'beverages',
 'breakfast',
 'canned goods',
 'dairy eggs',
 'deli',
 'dry goods pasta',
 'frozen',
 'household',
 'meat seafood',
 'pantry',
 'produce',
 'snacks']

In [3]:
check_config = {
    1: 15,
    2: 25,
    3: 30,
    4: 30,
    5: 35,
    6: 30,
    7: 25,
}

In [4]:
def get_order_items(order_id):
    return order_id, list(map(str, df[df['order_id'] == order_id]['product_id'].tolist()))

def create_input_for_genome(layout, i, j, level):
    def tile_enum_to_int(tile):
        if tile.type.value == 'wall':
            return 0
        if tile.type.value == 'floor':
            return 1
        if tile.type.value == 'rack':
            return 2
        if tile.type.value == 'door':
            return 3
        if tile.type.value == 'cashier':
            return 4
        return 0

    def convert_items_to_int(items):
        ids = []
        if len(items) == 0:
            return [(-1, -1), (-1, -1), (-1, -1), (-1, -1)]
        for item in items:
            if item[0] == '':
                ids.append((-1, 0))
            else:
                ids.append((int(item[0]), int(item[1])))
        return ids

    tile_info = get_tile_info(layout, i, j)
    if tile_info is None:
        return None

    left_products = convert_items_to_int(tile_info['left_products'])
    right_products = convert_items_to_int(tile_info['right_products'])

    return [
        level,
        tile_info['dist_to_cashier'],
        tile_info['dist_to_exit'],
        tile_info['orientation'][0],
        tile_info['orientation'][1],
        left_products[0][0],
        left_products[0][1],
        left_products[1][0],
        left_products[1][1],
        left_products[2][0],
        left_products[2][1],
        left_products[3][0],
        left_products[3][1],
        right_products[0][0],
        right_products[0][1],
        right_products[1][0],
        right_products[1][1],
        right_products[2][0],
        right_products[2][1],
        right_products[3][0],
        right_products[3][1],
        tile_enum_to_int(layout[i - 1][j]),
        tile_enum_to_int(layout[i + 1][j]),
        tile_enum_to_int(layout[i][j - 1]),
        tile_enum_to_int(layout[i][j + 1]),
        tile_enum_to_int(layout[i - 1][j - 1]),
        tile_enum_to_int(layout[i + 1][j + 1]),
        tile_enum_to_int(layout[i - 1][j + 1]),
        tile_enum_to_int(layout[i + 1][j - 1]),
    ]


In [5]:
def random_layout(items_list):
    layout = Layout('./../data/layout 18x25_6.json')
    layout.set_item_list(items_list)
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(random.choice(layout.get_item_list()), (row, col), level=lev)
    return layout

In [6]:
df = pd.read_csv('./../data/datasets/ECommerce_consumer behaviour.csv')
df = df[['order_id', 'user_id', 'order_number', 'department', 'product_id', 'product_name']]
check_list = []

df = df[df['department'].isin(selected_categories)]


def get_order_items(order_id):
    order = df[df['order_id'] == order_id]
    is_in_category = order['department'].apply(lambda x: x in selected_categories)
    return order_id, order[is_in_category]['product_name'].unique().tolist()\

# Create check list
check_ids = df['order_id'].unique().tolist()
check_list = []
for check_id in tqdm(check_ids[:10000]):
    check = get_order_items(check_id)
    check_list.append(check)

def get_checks_of_specific_length(check_list, length):
    return [x for x in check_list if len(x[1]) == length]

def get_checks_of_specific_length_range(check_list, range_dict):
    # range dict: length - n_of_checks
    res = []
    for key in tqdm(range_dict.keys()):
        res += get_checks_of_specific_length(check_list, key)[:range_dict[key]]
    return res

tuned_checks = get_checks_of_specific_length_range(check_list, check_config)

# convert tuned checks item names to item ids
check_list = []
name_id_df = df[['product_name', 'product_id']].drop_duplicates()
# ids are not sequential numbers, so we need to map them to sequential numbers
name_id_df['product_id_norm'] = range(1, len(name_id_df) + 1)
for check in tuned_checks:
    check_list.append((check[0], [str(name_id_df[name_id_df['product_name'] == x]['product_id_norm'].values[0]) for x in check[1]]))

#convert layout items to item ids
str_items = name_id_df['product_id_norm'].unique().tolist()
str_items = [str(x) for x in str_items]
layout.set_item_list(str_items, reset_items=False)
for i in range(layout.shape[0]):
    for j in range(layout.shape[1]):
        if layout[i][j].type.name == 'RACK':
            items = layout[i][j].items
            for level in range(layout.get_max_rack_level()):
                item = items[level]
                new_item = name_id_df[name_id_df['product_name'] == item[0]]['product_id_norm'].values[0]
                layout.set_item_to_rack(str(new_item), (i, j), level=level)
layout = random_layout(str_items)
best = None

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

# Data preparation

idea - take a layout, generate all possible changes for every rack, evaluate them and feed it to decision tree: state-action as X, reward diff as Y

In [7]:
data = {
    "state": [],
    "action": [],
    "reward": [], 
    "info": []
}

In [8]:
curr_info, _ = thread_func(layout, check_list)
current_reward = calculate_score(curr_info, layout, check_list, SCORE_COEFFICIENTS)

In [9]:
layout.reset_item_count()
layout.reset_path_count()

In [10]:
_, layout = thread_func(layout, check_list)

In [11]:
from concurrent.futures import ProcessPoolExecutor

for i in trange(layout.shape[0]):
    for j in trange(layout.shape[1]):
        if layout[i][j].type.name == 'RACK':
            for level in range(layout.get_max_rack_level()):
                input_data = create_input_for_genome(layout, i, j, level)
                if input_data is None:
                    continue
                futures = []
                with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
                    for item in str_items:
                        l = layout.copy()
                        l.set_item_to_rack(item, (i, j), level=level)
                        futures.append(executor.submit(thread_func, l, check_list))
                        data['action'].append(item)
                        data['state'].append(input_data)
                    for future in futures:
                        new_info, _ = future.result()
                        new_reward = calculate_score(new_info, l, check_list, SCORE_COEFFICIENTS)
                        data['reward'].append(new_reward - current_reward)
                        data['info'].append(new_info)

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
for k in data.keys():
    print(k, len(data[k]))

state 0
action 0
reward 0
info 0


In [12]:
# save data to file
import pickle

In [13]:
#with open('./../data/datasets/data.pkl', 'wb') as f:
#    pickle.dump(data, f)

In [14]:
# load data from file
with open('./../data/datasets/data1.pkl', 'rb') as f:
    data = pickle.load(f)

In [15]:
data_df = pd.DataFrame(data)
data_df

,state,action,reward,info
0,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",1,0.000000,"{'missing_items': [], 'path': 11338, 'invalid'..."
1,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",2,0.000000,"{'missing_items': [], 'path': 11182, 'invalid'..."
2,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",3,-0.264085,"{'missing_items': [], 'path': 11320, 'invalid'..."
3,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",4,0.000000,"{'missing_items': [], 'path': 11476, 'invalid'..."
4,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",5,0.000000,"{'missing_items': [], 'path': 11346, 'invalid'..."
...,...,...,...,...
55659,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",94,0.880282,"{'missing_items': [], 'path': 11075, 'invalid'..."
55660,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",95,0.880282,"{'missing_items': [], 'path': 11194, 'invalid'..."
55661,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",96,0.880282,"{'missing_items': [], 'path': 11149, 'invalid'..."
55662,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",97,0.880282,"{'missing_items': [], 'path': 11124, 'invalid'..."


In [16]:
from sklearn.preprocessing import OneHotEncoder

#split state ito separate columns
state_df = pd.DataFrame(data_df['state'].tolist(), columns=['level', 'dist_to_cashier', 'dist_to_exit', 'orientation_x', 'orientation_y', 'left_product_1_id', 'left_product_1_count', 'left_product_2_id', 'left_product_2_count', 'left_product_3_id', 'left_product_3_count', 'left_product_4_id', 'left_product_4_count', 'right_product_1_id', 'right_product_1_count', 'right_product_2_id', 'right_product_2_count', 'right_product_3_id', 'right_product_3_count', 'right_product_4_id', 'right_product_4_count', 'tile_up', 'tile_down', 'tile_left', 'tile_right', 'tile_up_left', 'tile_down_right', 'tile_up_right', 'tile_down_left'])
# join 2 dfs
data_df = data_df.drop('state', axis=1)
data_df = data_df.join(state_df)

In [17]:
data_df

,action,reward,info,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,left_product_1_id,left_product_1_count,...,right_product_4_id,right_product_4_count,tile_up,tile_down,tile_left,tile_right,tile_up_left,tile_down_right,tile_up_right,tile_down_left
0,1,0.000000,"{'missing_items': [], 'path': 11338, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
1,2,0.000000,"{'missing_items': [], 'path': 11182, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
2,3,-0.264085,"{'missing_items': [], 'path': 11320, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
3,4,0.000000,"{'missing_items': [], 'path': 11476, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
4,5,0.000000,"{'missing_items': [], 'path': 11346, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,94,0.880282,"{'missing_items': [], 'path': 11075, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0
55660,95,0.880282,"{'missing_items': [], 'path': 11194, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0
55661,96,0.880282,"{'missing_items': [], 'path': 11149, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0
55662,97,0.880282,"{'missing_items': [], 'path': 11124, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0


In [18]:
from sklearn.utils import shuffle
data_df = shuffle(data_df)

In [19]:
data_df

,action,reward,info,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,left_product_1_id,left_product_1_count,...,right_product_4_id,right_product_4_count,tile_up,tile_down,tile_left,tile_right,tile_up_left,tile_down_right,tile_up_right,tile_down_left
5566,79,0.000000,"{'missing_items': [], 'path': 10979, 'invalid'...",0,26,20,1,0,15,10,...,-1,-1,0,1,2,0,0,2,0,1
4121,6,0.264085,"{'missing_items': [], 'path': 10995, 'invalid'...",2,22,20,1,0,1,10,...,48,10,0,1,2,2,0,1,0,1
5489,2,0.000000,"{'missing_items': [], 'path': 11139, 'invalid'...",0,26,20,1,0,15,10,...,-1,-1,0,1,2,0,0,2,0,1
40338,61,0.000000,"{'missing_items': [], 'path': 11073, 'invalid'...",3,14,8,0,1,1,10,...,43,10,2,2,2,1,2,1,1,2
45980,19,0.000000,"{'missing_items': [], 'path': 11047, 'invalid'...",1,21,13,-1,0,-1,-1,...,56,10,1,2,2,1,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48103,84,0.880282,"{'missing_items': [], 'path': 11228, 'invalid'...",2,5,9,1,0,-1,-1,...,26,10,2,1,1,2,1,1,2,1
15357,70,0.000000,"{'missing_items': [], 'path': 11037, 'invalid'...",0,18,16,0,-1,1,10,...,86,10,2,2,1,2,1,2,2,1
3079,42,0.880282,"{'missing_items': [], 'path': 11014, 'invalid'...",3,19,21,1,0,1,10,...,78,10,0,1,2,2,0,1,0,1
47527,96,0.616197,"{'missing_items': [], 'path': 11158, 'invalid'...",0,4,12,1,0,72,10,...,-1,-1,2,1,2,1,2,1,1,1


In [20]:
X = np.array(data_df.drop(['reward','info'], axis=1).values.tolist())
y = np.array(data_df['reward'].values.tolist())

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_squared_error
import os

In [23]:
def run_regression(train_x, train_y, test_x, test_y, model, model_name, max_depth):
    model.fit(train_x, train_y)
    train_score = model.score(train_x, train_y)
    test_score = model.score(test_x, test_y)
    y_pred = model.predict(test_x)
    mse = mean_squared_error(test_y, y_pred)
    
    print(f'\'{model_name}\' --- depth: {max_depth}; train score: {train_score}; test score: {test_score}; mse: {mse}')
    
    export_graphviz(model, out_file=f'./../data/results/tree-{model_name}-{max_depth}.dot', feature_names=['level', 'dist_to_cashier', 'dist_to_exit', 'orientation_x', 'orientation_y', 'left_product_1_id', 'left_product_1_count', 'left_product_2_id', 'left_product_2_count', 'left_product_3_id', 'left_product_3_count', 'left_product_4_id', 'left_product_4_count', 'right_product_1_id', 'right_product_1_count', 'right_product_2_id', 'right_product_2_count', 'right_product_3_id', 'right_product_3_count', 'right_product_4_id', 'right_product_4_count', 'tile_up', 'tile_down', 'tile_left', 'tile_right', 'tile_up_left', 'tile_down_right', 'tile_up_right', 'tile_down_left', 'action'], filled=True)
    # convert dot file to png
    os.system(f'dot -Tpng ./../data/results/tree-{model_name}-{max_depth}.dot -o ./../data/results/tree-{model_name}-{max_depth}.png')
    return model

In [24]:
for max_depth in [2, 3, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]:
    model = DecisionTreeRegressor(max_depth=max_depth)
    run_regression(X_train, y_train, X_test, y_test, model, 'decision_tree', max_depth)

'decision_tree' --- depth: 2; train score: 0.059481668706430124; test score: 0.06205418819242137; mse: 1.2201690071016824
'decision_tree' --- depth: 3; train score: 0.1080522746670265; test score: 0.11713767400183972; mse: 1.1485111764022198
'decision_tree' --- depth: 5; train score: 0.26283945834084566; test score: 0.26218746676947036; mse: 0.959816627747533
'decision_tree' --- depth: 10; train score: 0.879635298319562; test score: 0.8441222261775823; mse: 0.20278061495639935


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.357777 to fit


'decision_tree' --- depth: 15; train score: 0.9811874853622149; test score: 0.9148554531507215; mse: 0.11076411438843364


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.112701 to fit


'decision_tree' --- depth: 20; train score: 0.9966689250448887; test score: 0.9110199757743895; mse: 0.11575366769004806


...

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.079164 to fit


'decision_tree' --- depth: 45; train score: 1.0; test score: 0.9185201991562071; mse: 0.10599666467172167


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.079701 to fit


'decision_tree' --- depth: 50; train score: 1.0; test score: 0.9185853406715524; mse: 0.10591192240079367


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.0787767 to fit


In [54]:
model = DecisionTreeRegressor(max_depth=10)

In [55]:
# fit model
model = run_regression(X_train, y_train, X_test, y_test, model, 'decision_tree', 30)

'decision_tree' --- depth: 30; train score: 0.879635298319562; test score: 0.8441484432966946; mse: 0.20274650923750423


dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.359051 to fit


In [56]:
# run model for each item in the layout

def generate_data_for_layout(layout):
    data = {
        "state": [],
        "action": [],
        "reward": [],
        "info": [],
        "coords": []
    }
    
    for i in range(layout.shape[0]):
        for j in range(layout.shape[1]):
            if layout[i][j].type.name == 'RACK':
                for level in range(layout.get_max_rack_level()):
                    input_data = create_input_for_genome(layout, i, j, level)
                    if input_data is None:
                        continue
                    for item in str_items:
                        data['action'].append(item)
                        data['state'].append(input_data)
                        data['info'].append(None)
                        data['reward'].append(None)
                        data['coords'].append((i, j, level))
    df = pd.DataFrame(data)
    state_df = pd.DataFrame(df['state'].tolist(), columns=['level', 'dist_to_cashier', 'dist_to_exit', 'orientation_x', 'orientation_y', 'left_product_1_id', 'left_product_1_count', 'left_product_2_id', 'left_product_2_count', 'left_product_3_id', 'left_product_3_count', 'left_product_4_id', 'left_product_4_count', 'right_product_1_id', 'right_product_1_count', 'right_product_2_id', 'right_product_2_count', 'right_product_3_id', 'right_product_3_count', 'right_product_4_id', 'right_product_4_count', 'tile_up', 'tile_down', 'tile_left', 'tile_right', 'tile_up_left', 'tile_down_right', 'tile_up_right', 'tile_down_left'])
    # join 2 dfs
    df = df.drop('state', axis=1)
    df = df.join(state_df)
    return df

In [57]:
data = generate_data_for_layout(layout)
data

,action,reward,info,coords,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,left_product_1_id,...,right_product_4_id,right_product_4_count,tile_up,tile_down,tile_left,tile_right,tile_up_left,tile_down_right,tile_up_right,tile_down_left
0,1,None,None,"(1, 2, 0)",0,18,28,1,0,-1,...,37,10,0,1,0,2,0,1,0,2
1,2,None,None,"(1, 2, 0)",0,18,28,1,0,-1,...,37,10,0,1,0,2,0,1,0,2
2,3,None,None,"(1, 2, 0)",0,18,28,1,0,-1,...,37,10,0,1,0,2,0,1,0,2
3,4,None,None,"(1, 2, 0)",0,18,28,1,0,-1,...,37,10,0,1,0,2,0,1,0,2
4,5,None,None,"(1, 2, 0)",0,18,28,1,0,-1,...,37,10,0,1,0,2,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,94,None,None,"(16, 22, 3)",3,19,11,-1,0,-1,...,32,5,1,0,2,0,1,0,2,0
55660,95,None,None,"(16, 22, 3)",3,19,11,-1,0,-1,...,32,5,1,0,2,0,1,0,2,0
55661,96,None,None,"(16, 22, 3)",3,19,11,-1,0,-1,...,32,5,1,0,2,0,1,0,2,0
55662,97,None,None,"(16, 22, 3)",3,19,11,-1,0,-1,...,32,5,1,0,2,0,1,0,2,0


In [58]:
# fit without reward and coords
X = np.array(data.drop(['reward', 'coords', 'info'], axis=1).values.tolist())

#predict rewards
y = model.predict(X)

#add rewards to data
data['reward'] = y

In [59]:
# for each item in layout, select the one with the highest reward
def select_best_item_for_layout(layout, data, rewards):
    new_layout = layout.copy()
    for i in trange(layout.shape[0]):
        for j in range(layout.shape[1]):
            if layout[i][j].type.name == 'RACK':
                for level in range(layout.get_max_rack_level()):
                    coords = (i, j, level)
                    coords_df = data[data['coords'] == coords]
                    #print(decoded_df)
                    coords_df.sort_values(by='reward', ascending=False)
                    #(coords_df)
                    item = coords_df.iloc[random.randint(0, 4)]['action']
                    #print(item, type(item))
                    new_layout.set_item_to_rack(item, (i, j), level=level)
    return new_layout

In [60]:
layout = layout.reset_item_count().reset_path_count().copy()

In [61]:
better_layout = layout.copy()

In [62]:
for i  in range(1):
    better_layout = better_layout.reset_item_count().reset_path_count()
    better_layout = select_best_item_for_layout(better_layout, data, y)
    res, l = thread_func(better_layout, check_list)
    print(res)

  0%|          | 0/18 [00:00<?, ?it/s]

{'missing_items': [['28'], ['41'], ['19'], ['33'], ['9'], ['9'], ['25'], ['33'], ['8'], ['20'], ['9'], ['48'], ['9'], ['36'], ['28'], ['44', '9'], ['8', '16'], ['30', '20'], ['43'], ['36', '20'], ['12', '68'], ['41', '26'], ['32', '28'], ['59'], ['36', '9'], ['9'], ['10', '36'], ['28', '19'], ['41', '48'], ['12', '71'], ['12', '10'], ['33', '9'], ['44', '48'], ['69'], ['9'], ['9', '44'], ['68'], ['68', '10'], ['14', '52'], ['33'], ['15', '19'], ['28', '27', '58'], ['20', '9', '33'], ['19', '51'], ['27', '9', '20'], ['9', '44'], ['67', '11', '10'], ['44', '33', '9'], ['75', '9', '41'], ['32', '77', '78'], ['53'], ['21', '24', '78'], ['9', '44'], ['34', '41'], ['10', '49'], ['17'], ['69', '16', '54'], ['10', '9', '68'], ['89', '10', '23'], ['45', '61', '25'], ['32', '83', '78'], ['53', '21'], ['90', '19', '17'], ['9', '8', '33'], ['78', '53', '9'], ['30', '66', '10'], ['20', '19', '30'], ['26', '9'], ['22', '9'], ['30', '20', '28', '58'], ['36', '9', '66', '10'], ['11', '36', '28', '9'],

In [63]:
eval = thread_func(layout, check_list)
test = thread_func(better_layout, check_list)

In [64]:
eval

({'missing_items': [['28'],
   ['28'],
   ['28'],
   ['28'],
   ['28'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9', '28'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['9', '28'],
   ['9'],
   ['9'],
   ['9'],
   ['28'],
   ['9'],
   ['9'],
   ['9'],
   ['9'],
   ['4'],
   ['4', '9'],
   ['4'],
   ['9'],
   ['9'],
   ['9', '4']],
  'path': 7181,
  'invalid': 41,
  'rack_uniformity': 976,
  'tile_uniformity': 562},
 <ui.python.Layout.Layout at 0x7fa14321ab00>)

In [65]:
test

({'missing_items': [['28'],
   ['41'],
   ['19'],
   ['33'],
   ['9'],
   ['9'],
   ['25'],
   ['33'],
   ['8'],
   ['20'],
   ['9'],
   ['48'],
   ['9'],
   ['36'],
   ['28'],
   ['44', '9'],
   ['8', '16'],
   ['30', '20'],
   ['43'],
   ['36', '20'],
   ['12', '68'],
   ['41', '26'],
   ['32', '28'],
   ['59'],
   ['36', '9'],
   ['9'],
   ['10', '36'],
   ['28', '19'],
   ['41', '48'],
   ['12', '71'],
   ['12', '10'],
   ['33', '9'],
   ['44', '48'],
   ['69'],
   ['9'],
   ['9', '44'],
   ['68'],
   ['68', '10'],
   ['14', '52'],
   ['33'],
   ['15', '19'],
   ['28', '27', '58'],
   ['20', '9', '33'],
   ['19', '51'],
   ['27', '9', '20'],
   ['9', '44'],
   ['67', '11', '10'],
   ['44', '33', '9'],
   ['75', '9', '41'],
   ['32', '77', '78'],
   ['53'],
   ['21', '24', '78'],
   ['9', '44'],
   ['34', '41'],
   ['10', '49'],
   ['17'],
   ['69', '16', '54'],
   ['10', '9', '68'],
   ['89', '10', '23'],
   ['45', '61', '25'],
   ['32', '83', '78'],
   ['53', '21'],
   ['90', '19'

In [66]:
dir = str(Path(os.getcwd()).parent)

In [68]:
better_layout.display_in_window(home_dir=dir)